In [1]:
import numpy as np
import keras
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [54]:
x = "0123456789"*100000

print len(x)

1000000


In [55]:
dct = ['*'] + list(set(x))
max_features = len(dct)
rev_dct = [(j, i) for i, j in enumerate(dct)]
rev_dct = dict(rev_dct)

In [56]:
print dct
print rev_dct

['*', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8']
{'*': 0, '1': 1, '0': 2, '3': 3, '2': 4, '5': 5, '4': 6, '7': 7, '6': 8, '9': 9, '8': 10}


In [57]:
x = [rev_dct[ch] for ch in x]
print len(x)

1000000


In [59]:
n_timestamps = 200
x = x[:len(x) - len(x) % n_timestamps]
x = np.array(x, dtype='int32').reshape((-1, n_timestamps))
print x.shape

(5000, 200)


In [26]:
print x[0][:10], x.shape

[ 2  1  4  3  6  5  8  7 10  9] (5000, 200)


In [60]:
y = np.zeros((x.shape[0], x.shape[1], max_features), dtype='int32')
for i in np.arange(x.shape[0]):
    for j in np.arange(x.shape[1]):
        y[i, j, x[i, j]] = 1

In [66]:
x = np.roll(y, 1, axis=1)
x[:, 0, :] = 0
x[:, 0, 0] = 1

print x.shape
print y.shape

(5000, 200, 11)
(5000, 200, 11)


In [36]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(200,  max_features)))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(Dense(max_features))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 200, 256)          274432    
_________________________________________________________________
lstm_8 (LSTM)                (None, 200, 256)          525312    
_________________________________________________________________
lstm_9 (LSTM)                (None, 200, 256)          525312    
_________________________________________________________________
dense_3 (Dense)              (None, 200, 11)           2827      
_________________________________________________________________
activation_3 (Activation)    (None, 200, 11)           0         
Total params: 1,327,883.0
Trainable params: 1,327,883.0
Non-trainable params: 0.0
_________________________________________________________________


In [38]:
hist = model.fit(x, y, batch_size=64, nb_epoch=1, verbose=2)

Epoch 1/1
52s - loss: 0.1519


In [40]:
import numpy
def mnrnd(probs):
    rnd = numpy.random.random()
    for i in xrange(len(probs)):
        rnd -= probs[i]
        if rnd <= 0:
            return i
    return i

sentences = numpy.zeros((128, n_timestamps, max_features))
sentences[:, 0, 0] = 1

# Start sampling char-sequences. At each iteration i the probability over
# the i-th character of each sequences is computed. 
for i in numpy.arange(10):
    probs = model.predict_proba(sentences, verbose=2)[:,i,:]
    # Go over each sequence and sample the i-th character.
    for j in numpy.arange(len(sentences)):
        sentences[j, i+1, mnrnd(probs[j, :])] = 1
sentences = [sentence[1:].nonzero()[1] for sentence in sentences]

# Convert to readable text.
text = []
for sentence in sentences:
    text.append(''.join([dct[word] for word in sentence]))
print text


['1123456789', '0123456789', '6922345678', '9113456689', '0023456789', '0123456789', '9123456789', '0123456789', '2223456789', '0023456789', '0123456789', '4013345678', '1023456789', '0013456789', '0123456789', '1023456789', '2123456789', '1123456789', '2133456789', '*112345678', '3112456789', '0123456789', '1023556789', '0123456789', '2023456789', '0123456789', '1123456789', '0623456679', '9223456789', '0223456789', '0223456789', '5123345678', '1113456789', '0123456789', '1123456789', '1123456789', '0123456789', '0123456789', '3113456789', '0123456789', '1113456789', '1123456789', '0123456789', '1123456789', '6122345678', '2123456789', '8023445778', '0123456789', '0113456789', '5223355679', '0823455678', '1023456789', '0123456789', '1123456789', '0023456789', '0123456789', '0123456789', '8023455678', '0123456789', '0123456789', '9123456789', '3123456789', '1233456789', '0123456789', '1123456789', '8123456789', '8123455789', '1023456789', '1113456789', '0123456789', '0023456789', '0023

In [43]:
for ix in text[:20]:
    print ix

1123456789
0123456789
6922345678
9113456689
0023456789
0123456789
9123456789
0123456789
2223456789
0023456789
0123456789
4013345678
1023456789
0013456789
0123456789
1023456789
2123456789
1123456789
2133456789
*112345678
